In [1]:
from transformers import BartTokenizerFast
import os
import json
import regex as re
import polars as pl
import torch
import random

In [2]:
tokenizer = BartTokenizerFast.from_pretrained("/kaggle/input/nepbart-tokenizer/nepbart_tokenizer")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'BartTokenizerFast'.


In [3]:
random.choice([1, 2, 3])

2

In [4]:
def token_infilling(
    tokenized_sequence: torch.Tensor,
    mask_token_id: int,
    mask_probability: float = 0.15,
    list_special_tokens: list = [],
) -> str:
    # print(len(tokenized_sequence))
    # print(tokenized_sequence)
    # if len(tokenied_sequence) <= 10:
    span_length = int(torch.poisson(torch.tensor([3.0])))
    perturbed_ids = torch.empty(0, dtype=torch.long)
    ## If span_length is found to be 0, make it 1, 2, or 3 randomly
    if span_length == 0:
        span_length = random.choice([1, 2, 3])
    # print(span_length)
    for i in range(0, len(tokenized_sequence), span_length):
        mask_pl = torch.rand(1)
        # print(mask_pl)
        if mask_pl < mask_probability:
            # check if the span does not contain special tokens
            # print([token in list_special_tokens for token in tokenized_sequence[i : i + span_length]])
            if not any([token in list_special_tokens for token in tokenized_sequence[i : i + span_length]]):
                perturbed_ids = torch.cat(
                    (perturbed_ids, torch.tensor([mask_token_id], dtype=torch.long))
                )
        else:
            perturbed_ids = torch.cat(
                (perturbed_ids, tokenized_sequence[i : i + span_length])
            )
    if mask_token_id not in perturbed_ids:
        return token_infilling(
            tokenized_sequence,
            mask_token_id,
            mask_probability,
            list_special_tokens,
        )
                
    # print(len(perturbed_ids))
    return perturbed_ids


In [5]:
# tokenizer("महायज्ञ गर्न दलित परिवारको उठीबास लगाएको घटनामा संवेदनशील भएर कार्य नगरेको भन्दै गृह महायज्ञ गर्न दलित परिवारको उठीबास लगाएको घटनामा  भएर कार्य नगरेको भन्दै गृह महायज्ञ गर्न दलित परिवारको उठीबास लगाएको घटनामा संवेदनशील भएर कार्य नगरेको भन्दै गृह महायज्ञ गर्न दलित परिवारको उठीबास लगाएको", return_tensors='pt', max_length=100, padding='max_length')

In [6]:
# token_infilling(tokenizer("महायज्ञ गर्न दलित परिवारको उठीबास लगाएको घटनामा संवेदनशील भएर कार्य नगरेको भन्दै गृह महायज्ञ गर्न दलित परिवारको उठीबास लगाएको घटनामा  भएर कार्य नगरेको भन्दै गृह महायज्ञ गर्न दलित परिवारको उठीबास लगाएको घटनामा संवेदनशील भएर कार्य नगरेको भन्दै गृह महायज्ञ गर्न दलित परिवारको उठीबास लगाएको", return_tensors='pt')['input_ids'][0], tokenizer.mask_token_id, list_special_tokens=tokenizer.all_special_ids)

In [7]:
train = pl.read_parquet('/kaggle/input/nepbart-llm-dataset/nepbart_dataset/nepbart_test.parquet')

In [8]:
SEQUENCE_LENGTH = 768

In [9]:
len(train)

218326

In [ ]:
labels = []
for row in train.iter_rows(named=True):
    tokenized = tokenizer(row['text'], return_tensors='pt')['input_ids'][0]

    for i in range(0, len(tokenized), SEQUENCE_LENGTH - 2):
        if len(tokenized[i:i+SEQUENCE_LENGTH - 2]) < 10:
            continue
        
        labels.append(tokenizer.decode(tokenized[i:i+SEQUENCE_LENGTH - 2], skip_special_tokens=True))

In [ ]:
len(labels)

In [ ]:
final_df = pl.DataFrame({
    'text': labels
})

In [ ]:
len(final_df)

In [ ]:
final_df.write_parquet('/kaggle/working/nepbart_chunked_test.parquet')